# Video
## Duke Kunshan University Fall 2017
### David Brady
Monday 20 November 2017

This is a the week four Jupyter Notebook for discussion in class



In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import subprocess
import skimage
import skimage.io as imio
import skimage.transform as trans
from skimage.viewer import ImageViewer
import glob
import re
from moviepy.editor import *
from time import gmtime, strftime, sleep
from scipy import interpolate as interp
import sys
sys.path.append('/usr/local/opt/opencv@3/lib/python3.6/site-packages')
import cv2
%matplotlib inline
mpl.rcParams['figure.figsize'] = (16.0, 8.0) 

/Users/yeko/anaconda/lib/python3.6/site-packages/skimage/viewer/utils/core.py:10: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warn("Recommended matplotlib backend is `Agg` for full "


In [2]:
def grabVideo(sid=1, imres=8, cameraNumber=1, fps=30):
    # I will use the file name vidGrab.... for the image grabbed from the camera. First step is to get rid of previously grabbed images
    cmdA='ssh ubuntu@10.0.1.'+str(cameraNumber)
    cmdB='scp ubuntu@10.0.1.'+str(cameraNumber)
    subprocess.run('rm vidGrab*', \
                   stdout=subprocess.PIPE, shell=True)
    subprocess.run(cmdA+' "rm vidGrab*"', \
                   stdout=subprocess.PIPE, shell=True)
    # ok, now let's grab an video on the tegra
    cmD=cmdA+' "nvgstcapture-1.0 -m 2 --sensor-id '+str(sid)+' -A '\
            +'--capture-auto 1 --fpsRange="'+str(fps)+' '+str(fps)+'"'\
            +'--capture-time 30 --file-name vidGrab --image-res '\
            + str(imres)+'"'
    subprocess.run(cmD, stdout=subprocess.PIPE, shell=True)
    subprocess.run(cmdB+':vidGrab* .', \
                   stdout=subprocess.PIPE, shell=True)
    #ok, now let's find the file name of the image we just grabbed 
    # and transfered 
    pop=subprocess.run('ls',stdout=subprocess.PIPE, shell=True)
    rePop=pop.stdout
    rePop=rePop.decode("utf-8")
    fileName = re.search(r'vidGrab(.*)', rePop)
    fileName=fileName.group()
    pop=VideoFileClip(fileName)
    return pop

In [3]:
def gstVideo(sid=1, cameraNumber=1, captureMode=1, frames=1200, framerate=240, fileOut="test.mp4"):
        cmdA='ssh ubuntu@10.0.1.'+str(cameraNumber)
        cmdB='scp ubuntu@10.0.1.'+str(cameraNumber)
        cmD=cmdA+" /home/ubuntu/bin/grabVideo.py "+str(sid)+' '+str(frames)+' '+str(captureMode)
        a=subprocess.run(cmD, stdout=subprocess.PIPE, shell=True)
        subprocess.run(cmdB+':grabVid.h264 .', \
            stdout=subprocess.PIPE, shell=True)
        subprocess.run('ffmpeg -r '+str(framerate)+' -i grabVid.h264 -an -y -vcodec copy '+fileOut, \
            stdout=subprocess.PIPE, shell=True)
        return a.stdout

In [ ]:
clip=grabVideo(fps=30)
clip.write_videofile("test3.mp4")
ipython_display("./test3.mp4", width=240)

In [16]:
fOut="clock30.mp4"
clip=gstVideo(fileOut=fOut)

In [6]:
orig = cv.LoadImage("test3.mp4")
cv.Flip(orig, flipMode=-1)
cv.ShowImage('180_rotation', orig)
cv.WaitKey(0)

AttributeError: module 'cv2' has no attribute 'LoadImage'